In [14]:
import numpy as np
import pandas as pd
from bigtree import Node
from random import randint

In [15]:
#récupération de tout les traitements connus
#traitements = pd.read_excel("implementation_N/data/traitementTotal.xlsx")
#traitements

In [16]:
#interactions = pd.read_excel("implementation_N/data/Interaction+Code.xlsx")
#interactions

Dictionnaire de règles connues :

nous avons prélevés certaines règles du document pour réaliser nos premiers exemples

In [17]:
ruleBase ={ 'rule1': {
        'condition': ['acide_acetylsaclicylique=true', 'anti-inflammatoire_non_stéroïdiens'],
        'consequence': '3'
    },

    'rule2': {
        'condition': ['acide_acetylsaclicylique>=3g','acetazolamide=true'],
        'consequence': '2'
    },
    'rule3': {
        'condition': ['acetazolamide=true', 'carbamazepine=true'],
        'consequence': '1'
    },
    'rule4': {
        'condition': ['abiraterone=true', 'metoprolol=true',],
        'consequence': '1'
    },

    'rule5': {
        'condition': ['acide_folinique=true', 'fluorouracil=true',],
        'consequence': '1'
    },

}

#Liste de tout les traitements existants
treatmentBase =['acide_folinique=true', 'fluorouracil=true','abiraterone=true', 'metoprolol=true','acetazolamide=true', 'carbamazepine=true',
                'acide_acetylsaclicylique>=3g','anti-inflammatoire_non_stéroïdiens','acide_acetylsaclicylique=true']

In [18]:
#nombre de traitement maximum à prendre en compte pour le training set
t_max = 2

#nombre de traitement maximum dans le cas réel : data frame de test
j_max = 8

In [19]:
#colonne du data frame
colonneReal = []
colonneReal.append("Nom")
colonneReal.append("Prénom")

for  i in range(j_max):
    colonneReal.append('treatment' + str(i+1))
    colonneReal.append('dose'+ str(i+1))
    colonneReal.append('startDate'+ str(i+1))
    colonneReal.append('endDate'+ str(i+1))
    

In [20]:
colonne = []
for i in range(t_max):
    colonne.append('treatment' + str(i+1))
colonne.append("iatrogenie")

Création de nouveaux patients : caractère iatrogénique ou non

In [21]:
#Fonction qui crée une ligne du dataFrame
def patientCreation(posologyList, treatmentList, ruleBase):
    dataLine = []
    #mise en forme des faits pour la détection par inférence
    facts=treatmentList
    
    #nombre de traitement, cette taille est fixée dès le début de l'algo
    t=len(treatmentList)

    for treatment in treatmentList:
        dataLine.append(treatment)
    iatrogenic_stored = 0
    #détection d'une situation iatrogénique à partir des règles
    #Ajouter la détection de l'indice iatrogénique maximum - OK
    for rule in ruleBase:
        conditions_met = all(condition in facts for condition in ruleBase[rule]['condition'])
        if conditions_met & (int(ruleBase[rule]['consequence'])>iatrogenic_stored):
            #dataLine.append(ruleBase[rule]['consequence'])#Le niveau de risque iatrogénique
            iatrogenic_stored = int(ruleBase[rule]['consequence'])

    #si on ne trouve pas de corrélation dans la liste des règles connues
    
    if iatrogenic_stored==0:
        dataLine.append(0)#Aucun risque iatrogénique   
    else:
        dataLine.append(iatrogenic_stored)#Le niveau de risque iatrogénique

    
    return dataLine


In [22]:
#fonction qui crée le dataFrame qui simule une base de donnée contenant des patients de l'hopital
# n = taille du dataFrame souhaité
# j = nombre de traitements maximum dans la base
def realPatientFrameCreation(colonne,n,j):

    patientList = []

    for i in range(n):
        #liste des traitements de taille future t_max
        elementList=[]
        #implémentation éventuelle d'une posologie
        posology_possible = [['Weight=light'],['Weight=medium'],['Weight=fat']]
        #index aléatoire qui va permettre de simuler un certain nombre de traitements pris par le patient
        i_limit = randint(0,j-1)
        
        posology=[]
        #choix d'un nom prénom pour le patient : ici c'est un essai
        patientName = (["Ham","Slim"])
        elementList.extend(patientName)
       
        for i in range(i_limit):
            
            #indice aléatoire pour piocher des traitements
            k1 = randint(0,len(treatmentBase)-1)
            #indice aléatoire pour piocher une posologie
            k_pos = randint(0,2)

            treatment = treatmentBase[k1]
            #on choisit bien un traitement qui n'est pas déjà utilisé
            while (treatment in elementList):
                k1 = randint(0,len(treatmentBase)-1)
                treatment = treatmentBase[k1]

            posology = posology_possible[k_pos].copy()
            #traitement
            elementList.append(treatment)
            #dose
            elementList.append("0")
            #date début
            elementList.append("27/12/2030")
            #date de fin
            elementList.append("28/12/2030")
        #ajout de "none" pour remplir le data set jusqu'au nombre de colonnes maximum
        for i in range(i_limit,j):
            for z in range(4):
                elementList.append("-")
            
        
        #création du nouveau patient à partir de la liste des éléments nécessaires
        newPatient = elementList.copy()
        patientList.append(newPatient)
    
    #création du data frame à partir de la liste de faux patients créées
    patientFrame = pd.DataFrame(data = patientList, columns=colonne)
    return patientFrame

In [23]:
#fonction qui crée le dataFrame avec une pondération : nous l'utilisons pour créer notre DataFrame d'entrainement 
#un version totalement aléatoire fonctionne aussi
#version pour des data frame de j traitement, prend en compte plusieurs tailles de règles

#n_regle est le nombre de règle actuel dans notre base de test 
n_regle = len(ruleBase) 

def patientFrameCreationNewJ(colonne,n,j,false_proba = 0.5, true_proba = 0.5):

    
    patientList = []

    #liste des posologies possibles
    posology_possible = [['Weight=light'],['Weight=medium'],['Weight=fat']]

    for k in range(n_regle):
        #on sélectionne la règle en cours dans le dictionnaire
        regleEnCours = ruleBase["rule"+str(k+1)]["condition"]
        
        #taille de la règle
        nRegleEnCours= len(regleEnCours)

        #on copie la liste des traitements pour retirer les traitements inclus dans la règle
        treatmentBaseEnCours = treatmentBase.copy()      
        treatmentBaseEnCours = list(set(treatmentBaseEnCours)-set(regleEnCours))
        
        for i in range(int(false_proba*int(n/n_regle))):
            #liste des traitements de taille j
            treatmentList = []
           
            #on ajoute tout les traitements de la règle sauf le dernier pour falsifier le patient en remplissant avec d'autres traitements
            for i in range(nRegleEnCours-1):
                treatmentList.append(regleEnCours[i])

            #on implémente les différents traitements supplémentaires
            for z in range(j-(nRegleEnCours-1)):

                #indice aléatoire pour piocher des traitements

                k1 = randint(0,len(treatmentBaseEnCours)-1)
                k_pos = randint(0,2)
                
                treatment = treatmentBaseEnCours[k1]
                while (treatment in treatmentList):
                    k1 = randint(0,len(treatmentBaseEnCours)-1)
                    treatment = treatmentBaseEnCours[k1]

                posology = posology_possible[k_pos]
                treatmentList.append(treatment)

             
            #création du nouveau patient à partir de la fonction patientCreation qui détecte la présence d'une situation iatrogénique
            newPatientFalse = patientCreation(posology, treatmentList,ruleBase)
            patientList.append(newPatientFalse)


        #pour l'instant on test en rajoutant exactement les même traitements qui est choisi aléatoirement à chaque fois
        treatmentList = []
        
        for i in range(int(true_proba*int(n/n_regle))):
            treatmentList = []
            treatmentList = regleEnCours.copy()

            #on rajoute des éléments aléatoire en fin de liste de traitement
            for z in range(j-nRegleEnCours):
                
                
                #indice aléatoire pour piocher des traitements
                
                k1 = randint(0,len(treatmentBaseEnCours)-1)
                k_pos = randint(0,2)
                
                treatment = treatmentBaseEnCours[k1]
                while (treatment in treatmentList):
                    k1 = randint(0,len(treatmentBaseEnCours)-1)
                    treatment = treatmentBaseEnCours[k1]

                posology = posology_possible[k_pos]
                treatmentList.append(treatment)
            
            k_pos = randint(0,2)
            posology = posology_possible[k_pos]
            newPatientTrue = patientCreation(posology, treatmentList,ruleBase)
            patientList.append(newPatientTrue)

        #création du nouveau patient à partir de la fonction précédente    
    
    patientFrame = pd.DataFrame(data = patientList, columns=colonne)
    
    #On mélange le dataFrame pour un aspect visuel plus réaliste
    patientFrame = patientFrame.sample(frac=1).reset_index(drop=True)

    return patientFrame

In [24]:
#DataSet d'entrainement: 
test_patientFrame = patientFrameCreationNewJ(colonne, 500,t_max)
test_patientFrame

,treatment1,treatment2,iatrogenie
0,acetazolamide=true,abiraterone=true,0
1,acide_acetylsaclicylique>=3g,abiraterone=true,0
2,acide_acetylsaclicylique=true,abiraterone=true,0
3,acide_acetylsaclicylique>=3g,metoprolol=true,0
4,acide_acetylsaclicylique=true,fluorouracil=true,0
...,...,...,...
495,acetazolamide=true,anti-inflammatoire_non_stéroïdiens,0
496,acide_folinique=true,fluorouracil=true,1
497,acide_acetylsaclicylique=true,anti-inflammatoire_non_stéroïdiens,3
498,acetazolamide=true,carbamazepine=true,1


In [25]:
#DataSet qui fait office de simulation de la base de donnée de patients
real_patient_frame = realPatientFrameCreation(colonneReal,20,8)
real_patient_frame

,Nom,Prénom,treatment1,dose1,startDate1,endDate1,treatment2,dose2,startDate2,endDate2,...,startDate6,endDate6,treatment7,dose7,startDate7,endDate7,treatment8,dose8,startDate8,endDate8
0,Ham,Slim,acide_acetylsaclicylique=true,0,27/12/2030,28/12/2030,acide_folinique=true,0,27/12/2030,28/12/2030,...,27/12/2030,28/12/2030,-,-,-,-,-,-,-,-
1,Ham,Slim,acide_acetylsaclicylique>=3g,0,27/12/2030,28/12/2030,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,Ham,Slim,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,Ham,Slim,acetazolamide=true,0,27/12/2030,28/12/2030,acide_folinique=true,0,27/12/2030,28/12/2030,...,-,-,-,-,-,-,-,-,-,-
4,Ham,Slim,acide_folinique=true,0,27/12/2030,28/12/2030,acide_acetylsaclicylique>=3g,0,27/12/2030,28/12/2030,...,-,-,-,-,-,-,-,-,-,-
5,Ham,Slim,carbamazepine=true,0,27/12/2030,28/12/2030,acide_acetylsaclicylique=true,0,27/12/2030,28/12/2030,...,27/12/2030,28/12/2030,fluorouracil=true,0,27/12/2030,28/12/2030,-,-,-,-
6,Ham,Slim,anti-inflammatoire_non_stéroïdiens,0,27/12/2030,28/12/2030,abiraterone=true,0,27/12/2030,28/12/2030,...,-,-,-,-,-,-,-,-,-,-
7,Ham,Slim,anti-inflammatoire_non_stéroïdiens,0,27/12/2030,28/12/2030,acide_folinique=true,0,27/12/2030,28/12/2030,...,-,-,-,-,-,-,-,-,-,-
8,Ham,Slim,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9,Ham,Slim,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [26]:
#Nettoyage des données inutiles

test_patientFrame.to_csv("patientFrame_N.csv")

real_patient_frame.to_csv("realFrame_N.csv")